In [ ]:
import numpy as np
import pandas as pd

import os, urllib, time
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
import mlflow, git
mlflow_client = mlflow.tracking.MlflowClient()

In [ ]:
NJ = 4
experiment_id = "0"
run_name = f"eos.{NJ}J"

rank_metrics = ['|alpha_tmu|', '|eyeerr_tmu|', '-reward_total']
print_metrics = ['model_version', 'alpha_tmu', 'alpha_tsigma', 'alpha_tdf', 'eyeerr_tmu', 'eyeerr_tsigma', 'eyeerr_tdf', 'reward_total']

In [ ]:
parent_runs = mlflow_client.search_runs(experiment_id, f"tags.mlflow.runName='{run_name}'")

best_metric = dict()
best_run = dict()

for parent_run in parent_runs:
    child_runs = mlflow_client.search_runs(experiment_id, f"tags.mlflow.parentRunId='{parent_run.info.run_id}'")
    #                                           order_by=["metrics.reward_total DESC"])
    for child_run in child_runs:
        if 'alpha_tmu' not in child_run.data.metrics:
            continue
            
        child_run.data.metrics['|alpha_tmu|'] = np.abs(child_run.data.metrics['alpha_tmu'])
        child_run.data.metrics['|eyeerr_tmu|'] = np.abs(child_run.data.metrics['eyeerr_tmu'])
        child_run.data.metrics['-reward_total'] = - child_run.data.metrics['reward_total']
        
        for metric in rank_metrics:
            if metric not in best_metric or child_run.data.metrics[metric] < best_metric[metric]:
                best_metric[metric] = child_run.data.metrics[metric]
                best_run[metric] = child_run

In [ ]:
metric_values = []
for whats_best, run in best_run.items():
    best_values = [run.data.metrics[metric] for metric in print_metrics]
    metric_values.append([whats_best] + best_values)
pd.DataFrame(metric_values, columns=['whats_best'] + print_metrics)    